In [1]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [8]:
products_file = "./ProdottiKebab/ProdottiVecchi.json"
orders_dir = "./OrdiniKebab"

In [9]:
products_df = pd.read_json(products_file)
products_df

,name,category,price,ingredients
0,marinara,pizza,3.8,"[pomodoro, origano, aglio, olio]"
1,marinara,pizza famiglia,8.6,"[pomodoro, origano, aglio, olio]"
2,margherita,pizza,4.0,"[pomodoro, mozzarella]"
3,margherita,pizza famiglia,9.0,"[pomodoro, mozzarella]"
4,romana,pizza,5.0,"[pomodoro, mozzarella, acciughe, origano]"
...,...,...,...,...
184,sole,pizza famiglia,16.0,"[pomodoro, mozzarella, asparagi, grana, uovo]"
185,napoletana,pizza fritta,6.5,"[scamorza, friarielli]"
186,pizza fritta,pizza fritta,6.5,[]
187,leggera,pizza fritta,6.5,"[philadelfia, pomodorini, mozzarella]"


In [10]:
unsafe_categories = ['pizza', 'pizza famiglia']
kebab_products = ['kebab panino', 'kebab piadina', 'kebab panino menu', 'kebab piadina menu', 'piatto piccolo',
                 'piatto grande', 'pizza kebab', 'piatto falafel', 'panino pizza']
sold_products = []
orders_count = []
orders_total = []
for order_file in os.listdir(orders_dir):
    if "09-2022" not in order_file:
        all_orders = pd.read_json(orders_dir + "/" + order_file)
        
        orders_count.append([order_file.replace(".json", ""), len(all_orders)])
        
        orders = all_orders['orders']
        for order in orders:
            order_name = order['name']
            order_amount = order['actualTotal']
            order_extra_price = order['extraPrice']
            order_type = order['type']
            if order_type != 'DELIVERY':
                # compute the total manually
                order_products = order['products']
                total = 0
                for product in order_products:
                    product_name = product['name']
                    product_category = product['category']
                    
                    like_category = product_category
                    if like_category == "pizza famiglia 2 gusti" or like_category == "pizza famiglia 3 gusti" or \
                    like_category == "pizza famiglia 4 gusti":
                        like_category = "pizza famiglia"

                    product_row = products_df[(products_df['name'] == product_name) &
                                          (products_df['category'] == like_category)]
                    if len(product_row) == 0:
                        print(order_file, order_name, product_name, len(product_row))
                    
                    product_ingredients = []
                    personalized = False
                    if product_name not in kebab_products:
                        for ingredient in product['ingredients']:
                            product_ingredients.append(ingredient)
                            if product_category in unsafe_categories:
                                total += 1
                                personalized = True
                        for ingredient in product_row['ingredients']:
                            product_ingredients.append(ingredient)
                    else:
                        if "tutto" in product['ingredients']:
                            for ingredient in product_row['ingredients']:
                                product_ingredients.append(ingredient)
                            for ingredient in product['ingredients'][1:]:
                                product_ingredients.append(ingredient)
                        else:
                            for ingredient in product['ingredients']:
                                product_ingredients.append(ingredient)
                    
                    product_price = product_row['price']
                    total += (product_price * product['quantity'])
                    sold_products.append([order_file.replace(".json", ""), order_name, product_name, product_ingredients,
                                         personalized])
                    
                orders_total.append([order_file.replace(".json", ""), order_name, total])
            else:
                if order_amount > 0:
                    order_products = order['products']
                    for product in order_products:
                        product_name = product['name']
                        product_category = product['category']
                        
                        like_category = product_category
                        if like_category == "pizza famiglia 2 gusti" or like_category == "pizza famiglia 3 gusti" or \
                        like_category == "pizza famiglia 4 gusti":
                            like_category = "pizza famiglia"

                        product_row = products_df[(products_df['name'] == product_name) &
                                              (products_df['category'] == like_category)]
                        if len(product_row) == 0:
                            print(order_file, order_name, product_name, len(product_row))

                        product_ingredients = []
                        personalized = False
                        if product_name not in kebab_products:
                            for ingredient in product['ingredients']:
                                product_ingredients.append(ingredient)
                                if product_category in unsafe_categories:
                                    personalized = True
                            for ingredient in product_row['ingredients']:
                                product_ingredients.append(ingredient)
                        else:
                            if "tutto" in product['ingredients']:
                                for ingredient in product_row['ingredients']:
                                    product_ingredients.append(ingredient)
                                for ingredient in product['ingredients'][1:]:
                                    product_ingredients.append(ingredient)
                            else:
                                for ingredient in product['ingredients']:
                                    product_ingredients.append(ingredient)
                        sold_products.append([order_file.replace(".json", ""), order_name, product_name, product_ingredients,
                                             personalized])
                            
                    orders_total.append([order_file.replace(".json", ""), order_name, order_amount + order_extra_price])
                else:
                    # compute the total manually
                    order_products = order['products']
                    total = 0
                    for product in order_products:
                        product_name = product['name']
                        product_category = product['category']

                        like_category = product_category
                        if like_category == "pizza famiglia 2 gusti" or like_category == "pizza famiglia 3 gusti" or \
                        like_category == "pizza famiglia 4 gusti":
                            like_category = "pizza famiglia"

                        product_row = products_df[(products_df['name'] == product_name) &
                                              (products_df['category'] == like_category)]
                        if len(product_row) == 0:
                            print(order_file, order_name, product_name, len(product_row))

                        product_ingredients = []
                        personalized = False
                        if product_name not in kebab_products:
                            for ingredient in product['ingredients']:
                                product_ingredients.append(ingredient)
                                if product_category in unsafe_categories:
                                    total += 1
                                    personalized = True
                            for ingredient in product_row['ingredients']:
                                product_ingredients.append(ingredient)
                        else:
                            if "tutto" in product['ingredients']:
                                for ingredient in product_row['ingredients']:
                                    product_ingredients.append(ingredient)
                                for ingredient in product['ingredients'][1:]:
                                    product_ingredients.append(ingredient)
                            else:
                                for ingredient in product['ingredients']:
                                    product_ingredients.append(ingredient)

                        product_price = product_row['price']
                        total += (product_price * product['quantity'])
                        sold_products.append([order_file.replace(".json", ""), order_name, product_name, product_ingredients,
                                             personalized])
                        
                    orders_total.append([order_file.replace(".json", ""), order_name, total])